# ***Fake News Classification Using Birectional LSTM RNN***

In [2]:
import pandas as pd

In [6]:
df = pd.read_csv('sample_data/fake_train.csv',engine='python',error_bad_lines=False)
df.head()

<ipython-input-6-e9f2b544c7d7>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('sample_data/fake_train.csv',engine='python',error_bad_lines=False)
Skipping line 9036: unexpected end of data


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.isnull().sum()

id          0
title     250
author    864
text       17
label       0
dtype: int64

In [8]:
df.shape

(9034, 5)

In [9]:
df = df.dropna() # we can't replace text data in null value position

In [10]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
df.shape

(7920, 5)

In [14]:
df.reset_index(inplace=True,drop=True)

In [15]:
df[0:10]

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
7,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
8,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
9,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


In [16]:
## get the independent features
X =df.drop('label',axis=1)

In [17]:
y = df['label']

In [18]:
## Check weather dataset is balanced or imbalenced
y.value_counts()

0    4510
1    3410
Name: label, dtype: int64

In [19]:
import tensorflow as tf

In [20]:
tf.__version__

'2.15.0'

In [21]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [22]:
## vocabulary size
voc_size=5000

In [23]:
messages = X.copy()

In [25]:
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [27]:
import nltk
import re
from nltk.corpus import stopwords


In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [32]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus =[]
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review =  review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [33]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [34]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
print(onehot_repr)

[[1720, 4579, 3119, 2910, 3756, 696, 127, 4616, 1314, 3332], [2794, 4789, 4511, 1935, 2197, 3314, 2041], [2829, 1651, 2837, 4488], [663, 352, 879, 4530, 1689, 4598], [656, 2197, 4715, 1118, 3073, 1034, 2197, 2533, 1891, 4139], [3260, 2441, 4883, 1391, 2564, 4076, 4152, 1037, 3154, 1557, 1833, 2744, 2970, 3935, 2041], [405, 932, 4668, 3908, 3959, 1588, 736, 4124, 3736, 51, 4788], [32, 1145, 426, 315, 976, 4753, 4076, 1449, 3736, 51, 4788], [963, 84, 416, 4910, 751, 4289, 2994, 3272, 4076, 1835], [2996, 200, 1255, 1790, 2375, 2048, 4989, 2319], [2434, 779, 3331, 225, 2099, 3705, 1650, 2361, 2324, 1558, 3821], [4530, 1186, 3756, 4289, 4076, 976], [3943, 2013, 3141, 2281, 3897, 4329, 2402, 3986, 4408], [3722, 3213, 4154, 3652, 2082, 1265, 4401, 3736, 51, 4788], [2340, 2025, 4930, 2573, 3266, 3736, 51, 4788], [4639, 4041, 2852, 1994, 881, 115, 3232, 2180, 4718, 696], [4106, 4187, 4789], [2818, 4686, 3492, 2121, 4076, 4829, 272, 2041], [4286, 81, 4511, 1414, 4550, 2083, 1106, 1276, 1034], [1

In [35]:
## Embedding Representation
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 4616, 1314, 3332],
       [   0,    0,    0, ..., 2197, 3314, 2041],
       [   0,    0,    0, ..., 1651, 2837, 4488],
       ...,
       [   0,    0,    0, ..., 3736,   51, 4788],
       [   0,    0,    0, ..., 3736,   51, 4788],
       [   0,    0,    0, ..., 3736,   51, 4788]], dtype=int32)

In [45]:
## creating the model
embedding_vector_features =40
model =Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer = 'adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 200)               112800    
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [46]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [47]:
## train_test_Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final,y_final, test_size =0.33,random_state=42)

In [48]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
166/166 [==============================] - 12s 41ms/step - loss: 0.3508 - accuracy: 0.8355 - val_loss: 0.1898 - val_accuracy: 0.9227
Epoch 2/10
166/166 [==============================] - 7s 43ms/step - loss: 0.1514 - accuracy: 0.9435 - val_loss: 0.2004 - val_accuracy: 0.9197
Epoch 3/10
166/166 [==============================] - 6s 36ms/step - loss: 0.0813 - accuracy: 0.9704 - val_loss: 0.2604 - val_accuracy: 0.9208
Epoch 4/10
166/166 [==============================] - 7s 42ms/step - loss: 0.0443 - accuracy: 0.9855 - val_loss: 0.3457 - val_accuracy: 0.9074
Epoch 5/10
166/166 [==============================] - 5s 32ms/step - loss: 0.0302 - accuracy: 0.9910 - val_loss: 0.3381 - val_accuracy: 0.9151
Epoch 6/10
166/166 [==============================] - 7s 44ms/step - loss: 0.0166 - accuracy: 0.9949 - val_loss: 0.3799 - val_accuracy: 0.9109
Epoch 7/10
166/166 [==============================] - 5s 32ms/step - loss: 0.0119 - accuracy: 0.9964 - val_loss: 0.4542 - val_accuracy: 0.913

In [49]:
y_pred = model.predict(X_test)


82/82 [==============================] - 2s 10ms/step


In [50]:
y_pred

array([[9.9995828e-01],
       [9.9998981e-01],
       [3.5947596e-06],
       ...,
       [8.2764989e-01],
       [3.7791821e-08],
       [2.6380325e-05]], dtype=float32)

In [51]:
y_pred>=0.5

array([[ True],
       [ True],
       [False],
       ...,
       [ True],
       [False],
       [False]])

In [53]:
y_pred = np.where(y_pred>=0.5,1,0)

In [55]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[1372,  122],
       [ 110, 1010]])

In [56]:
print(accuracy_score(y_test,y_pred))

0.9112471308339709
